<a href="https://colab.research.google.com/github/tubagokhan/ADGM/blob/main/RegulatorParserToJson.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [1]:
!pip install python-docx

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 239.6/239.6 kB 4.9 MB/s eta 0:00:00


In [5]:
from docx import Document
import json
import os
import glob

def extract_data_from_docx(docx_path):
    # Load the DOCX file
    doc = Document(docx_path)

    # Initialize variables
    section_id = 0
    subsection_id = 0
    context_id = 0
    current_section = ""
    current_subsection = ""
    current_context = ""
    data_structure = []
    context_level = 0
    heading_encountered = False  # Variable to track if a heading has been encountered

    # Process the document
    for para in doc.paragraphs:
        if para.style.name.startswith('Heading'):
            heading_level = int(para.style.name.split(' ')[-1])
            heading_encountered = True  # Set flag to True once a heading is encountered

            if heading_level <= context_level:
                # Append the last context before starting a new one
                data_structure.append({
                    "SectionID": str(section_id),
                    "SectionHeader": current_section,
                    "SubSectionID": f"{section_id}.{subsection_id}",
                    "SubSectionHeader": current_subsection,
                    "ContextID": f"{section_id}.{subsection_id}.{context_id}",
                    "Context": current_context.strip()
                })
                current_context = ""

            if 'Heading 1' in para.style.name:
                section_id += 1
                subsection_id = 0
                context_id = 0
                current_section = para.text.strip()
                context_level = 1

            elif 'Heading 2' in para.style.name:
                subsection_id += 1
                context_id = 0
                current_subsection = para.text.strip()
                context_level = 2

            elif 'Heading 3' in para.style.name:
                context_id += 1
                context_level = 3
                current_context += para.text + " "

            elif para.style.name != "Heading 1" and para.style.name != "Heading 2" and para.style.name != "Heading 3":
                current_context += para.text + " "

        elif heading_encountered and (para.style.name in ["UK11 Block 0.5", "UK11 Block 1.0", "Normal", "List Paragraph"] or context_level >= 3):
            # Accumulate text for the current context
            current_context += para.text + " "

    # Add the last context if it was not added
    if current_context:
        data_structure.append({
            "SectionID": str(section_id),
            "SectionHeader": current_section,
            "SubSectionID": f"{section_id}.{subsection_id}",
            "SubSectionHeader": current_subsection,
            "ContextID": f"{section_id}.{subsection_id}.{context_id}",
            "Context": current_context.strip()
        })

    return data_structure

def process_all_docx_in_folder(folder_path):
    for docx_file in glob.glob(os.path.join(folder_path, '*.docx')):
        extracted_data = extract_data_from_docx(docx_file)
        json_path = docx_file.rsplit('.', 1)[0] + '.json'
        with open(json_path, 'w') as json_file:
            json.dump(extracted_data, json_file, indent=4)
        print(f"Data extracted from {docx_file} and saved to JSON file: {json_path}")

# Specify the folder containing the DOCX files
folder_path = '/content/drive/Othercomputers/MBZUAI/MBZUAI/ADGM Project/ADGM Docs/FSRA Guidance'  # Replace with your folder path
process_all_docx_in_folder(folder_path)


Data extracted from /content/drive/Othercomputers/MBZUAI/MBZUAI/ADGM Project/ADGM Docs/FSRA Guidance/CMC_VER03.270922.docx and saved to JSON file: /content/drive/Othercomputers/MBZUAI/MBZUAI/ADGM Project/ADGM Docs/FSRA Guidance/CMC_VER03.270922.json
Data extracted from /content/drive/Othercomputers/MBZUAI/MBZUAI/ADGM Project/ADGM Docs/FSRA Guidance/Supplementary Guidance OTCLPs (VER02.181223).docx and saved to JSON file: /content/drive/Othercomputers/MBZUAI/MBZUAI/ADGM Project/ADGM Docs/FSRA Guidance/Supplementary Guidance OTCLPs (VER02.181223).json
Data extracted from /content/drive/Othercomputers/MBZUAI/MBZUAI/ADGM Project/ADGM Docs/FSRA Guidance/Guidance - Regulation of Spot Commodities Activities in ADGM (VER02.181223).docx and saved to JSON file: /content/drive/Othercomputers/MBZUAI/MBZUAI/ADGM Project/ADGM Docs/FSRA Guidance/Guidance - Regulation of Spot Commodities Activities in ADGM (VER02.181223).json
Data extracted from /content/drive/Othercomputers/MBZUAI/MBZUAI/ADGM Project